<br>

## __Exercício: Detecção de Anomalias__

<br>

__1:__

Utilizando a classe DetectorAnomalias criada ao longo do módulo, __vamos avaliar um detector de anomalias.__

O dataset utilizado pode ser importado através da função getData. 

Nesse conjunto de dados, possuímos 6 variáveis explicativas, $X_1, .., X_6$ e uma variável com a marcação se a instância é uma anomalia ou não.

Utilizando a __metodolodia__ discutida ao longo do módulo, __teste diferentes modelos (variando o limiar $\epsilon$)__ a fim de encontrar o que __melhor fita os dados.__

Justifique as escolhas do $\epsilon$, bem como quais as métricas de performance abordadas. 

<br>

__2:__ 

Aborde o problema num contexto de aprendizado supervisionado, ou seja, treine modelos de classificação binária com o objetivo de detectar anomalias.

Compare os resultados entre as metodologias.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd 
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
class DetectorAnomalias():
    
    def __init__(self, epsilon):
        self.epsilon = epsilon
        
    def fit(self, X):
        medias = X.mean(axis = 0)
        desvios = X.std(axis = 0)
        gaussianas = [st.norm(loc = m, scale = d) for m, d in zip(medias, desvios)]  
        self.gaussianas = gaussianas
        self.X = X
        
    def prob(self, x):
        p = 1
        for i in range(self.X.shape[1]):
            gaussiana_i = self.gaussianas[i]
            x_i = x[i]
            p *= gaussiana_i.pdf(x_i)
        return p
    
    def isAnomaly(self, x):
        return int(np.where(self.prob(x) < self.epsilon, 1, 0))

In [ ]:
def getData():
    return pd.read_csv("/content/drive/MyDrive/CIENTISTA DE DADOS/MENTORAMA - AULAS/MODULO 14/dataframe_anomalias_exercicio.csv")

In [ ]:
df = getData()
df

,x1,x2,x3,x4,x5,x6,anomalia
0,7.731153,23.299155,-0.367453,4.715372,9.306179,16.780965,0.0
1,11.466833,16.943695,-0.245131,7.060311,10.462826,19.821289,0.0
2,11.501272,20.196011,1.206049,-4.957189,7.771262,19.100079,0.0
3,10.893921,16.072385,2.738045,-3.684228,7.373334,23.225524,0.0
4,10.091706,19.253894,0.996895,-9.504052,8.883988,17.903298,0.0
...,...,...,...,...,...,...,...
10095,11.192286,18.451987,-0.953650,-14.362996,10.875826,17.056541,0.0
10096,12.014177,19.461815,1.985099,-7.119190,11.079922,17.582755,0.0
10097,10.745460,18.175951,0.206037,-1.897015,9.888329,17.963324,0.0
10098,9.893969,22.333270,-1.465981,4.137382,7.690620,21.570097,0.0


In [ ]:
df.anomalia.value_counts()

0.0    10046
1.0       54
Name: anomalia, dtype: int64

In [ ]:
#Dividindo o Dataset
X = df.drop(columns= 'anomalia').values
y = df['anomalia'].values

In [ ]:
#Buscando o melhor valor para o eps

eps = [1e-9,1e-8,1e-7,1e-6,1e-5,1e-3,1e-1]
scores=[]
for ep in eps:
    model = DetectorAnomalias(epsilon=ep)
    model.fit(X)
    pred=[]
    for i in range(len(X)):
        pred.append(model.isAnomaly(X[i, ]))

    fpr, tpr, thresholds = roc_curve(y, pred)
    scores.append(auc(fpr, tpr))
    print('using epsilon {} the auc scores was: {}'.format(ep, auc(fpr, tpr)))

using epsilon 1e-09 the auc scores was: 0.5925925925925926
using epsilon 1e-08 the auc scores was: 0.9996018315747561
using epsilon 1e-07 the auc scores was: 0.9837248656181564
using epsilon 1e-06 the auc scores was: 0.908222177981286
using epsilon 1e-05 the auc scores was: 0.6675293649213617
using epsilon 0.001 the auc scores was: 0.5
using epsilon 0.1 the auc scores was: 0.5


### Escolha do epsilon

O melhor eps foi de __1e-08__, resultando em auc score: 0.999, ou seja, o modelo está tendo previsões de 99,9% de acerto.

In [ ]:
#Aplicando o epsilon escolhido
modelo_eps = DetectorAnomalias(epsilon=1e-8)
modelo_eps.fit(X)
predito = []

for i in range(len(X)):
  predito.append(modelo_eps.isAnomaly(X[i, ]))

fpr, tpr, thresholds = roc_curve(y, predito)
scores.append(auc(fpr, tpr))
print('Auc score :', auc(fpr, tpr))

Auc score : 0.9996018315747561


In [ ]:
df['predito'] = predito
df.head()

,x1,x2,x3,x4,x5,x6,anomalia,predito
0,7.731153,23.299155,-0.367453,4.715372,9.306179,16.780965,0.0,0
1,11.466833,16.943695,-0.245131,7.060311,10.462826,19.821289,0.0,0
2,11.501272,20.196011,1.206049,-4.957189,7.771262,19.100079,0.0,0
3,10.893921,16.072385,2.738045,-3.684228,7.373334,23.225524,0.0,0
4,10.091706,19.253894,0.996895,-9.504052,8.883988,17.903298,0.0,0


In [ ]:
print("EPSILON = 1e-08")
print("=====" *10)

print("Valores real de anomalia")
print(df.anomalia.value_counts())
print("=====" *10)
print("Valores predito de anomalia")
print(df.predito.value_counts())

EPSILON = 1e-08
Valores real de anomalia
0.0    10046
1.0       54
Name: anomalia, dtype: int64
Valores predito de anomalia
0    10038
1       62
Name: predito, dtype: int64


__*OBS.: Foram encontrados 8 valores a mais do valor real com anomalias*__

In [ ]:
#Dividindo o Dataset em Treino, Validação e Teste
df1 = df[(df.anomalia !=0)]
df2 = df[(df.anomalia !=1)]

In [ ]:
treino = df2[0:6046]
val = pd.concat([df2[6046:8046], df1[0:27]])
test = pd.concat([df2[8046:10046],df1[27:54]])

In [ ]:
x_treino = treino.drop(columns='anomalia').values
y_treino = treino.anomalia.values

x_val = val.drop(columns='anomalia').values
y_val = val.anomalia.values

x_test = test.drop(columns='anomalia').values
y_test = test.anomalia.values

In [ ]:
#fitando o modelo utilizando detector de anomalias
modelo = DetectorAnomalias(epsilon = 1e-8)
modelo.fit(x_treino)

In [ ]:
#fazendo previsões com dados de validacao
y_pred_val =[]
for i in range(len(y_val)):
  y_pred_val.append(modelo.isAnomaly(x_val[i, ]))

fpr1, tpr1, thresholds = roc_curve(y_val, y_pred_val)
print('auc score:', auc(fpr1, tpr1))

auc score: 0.9994999999999999


In [ ]:
#Fazendo Previsões com dados de teste
y_pred_test = []
for i in range(len(y_test)):
  y_pred_test.append(modelo.isAnomaly(x_test[i, ]))

fpr2, tpr2, thresholds = roc_curve(y_test, y_pred_test)
print('auc score:', auc(fpr2, tpr2))

auc score: 0.9994999999999999


In [ ]:
#criando dataframe para compração de valores
dff1 = pd.DataFrame(x_val)
dff2 = pd.DataFrame(x_test)
                    
dff1['anomalia'] = y_val
dff2['anomalia'] = y_test

In [ ]:
dff1['anomalia_treino'] = y_pred_val
dff2['anomalia_test'] = y_pred_test

In [ ]:
print("VALIDAÇÃO")
print("==" *10)

print("Valores real de anomalia")
print(dff1.anomalia.value_counts())
print("==" *10)
print("Valores predito de anomalia")
print(dff1.anomalia_treino.value_counts())

VALIDAÇÃO
Valores real de anomalia
0.0    2000
1.0      27
Name: anomalia, dtype: int64
Valores predito de anomalia
0    1998
1      29
Name: anomalia_treino, dtype: int64


In [ ]:
print("TESTE")
print("==" *10)

print("Valores real de anomalia")
print(dff2.anomalia.value_counts())
print("==" *10)
print("Valores predito de anomalia")
print(dff2.anomalia_test.value_counts())

TESTE
Valores real de anomalia
0.0    2000
1.0      27
Name: anomalia, dtype: int64
Valores predito de anomalia
0    1998
1      29
Name: anomalia_test, dtype: int64


# Conclusão

Podemos observar que no modelo não supervisionado a detecção de anomalia considerou 8 valores a mais como anomalias escolhendo um valor de eps muito baixo

Já no método de detecção de anomalia supervisionado (classificação binaria), o erro na validação/teste foram de apenas 2 valores considerados com anomalia.